In [1]:
import os
import math
import time
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor

In [2]:
try:
    import lightgbm as lgb
    LGB_AVAILABLE = True
except Exception:
    LGB_AVAILABLE = False

In [3]:
train_data = pd.read_csv('/kaggle/input/playground-series-s5e9/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s5e9/test.csv')
sample = pd.read_csv('/kaggle/input/playground-series-s5e9/sample_submission.csv')

In [4]:
train_data.head()

,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
0,0,0.603610,-7.636942,0.023500,0.000005,0.000001,0.051385,0.409866,290715.6450,0.826267,147.53020
1,1,0.639451,-16.267598,0.071520,0.444929,0.349414,0.170522,0.651010,164519.5174,0.145400,136.15963
2,2,0.514538,-15.953575,0.110715,0.173699,0.453814,0.029576,0.423865,174495.5667,0.624667,55.31989
3,3,0.734463,-1.357000,0.052965,0.001651,0.159717,0.086366,0.278745,225567.4651,0.487467,147.91212
4,4,0.532968,-13.056437,0.023500,0.068687,0.000001,0.331345,0.477769,213960.6789,0.947333,89.58511


In [5]:
test_data.head()

,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy
0,524164,0.410013,-16.794967,0.023500,0.232910,0.012689,0.271585,0.664321,302901.5498,0.424867
1,524165,0.463071,-1.357000,0.141818,0.057725,0.257942,0.097624,0.829552,221995.6643,0.846000
2,524166,0.686569,-3.368928,0.167851,0.287823,0.210915,0.325909,0.304978,357724.0127,0.134067
3,524167,0.885793,-5.598049,0.118488,0.000005,0.376906,0.134435,0.487740,271790.3989,0.316467
4,524168,0.637391,-7.068160,0.126099,0.539073,0.068950,0.024300,0.591248,277728.5383,0.481067


In [6]:
sample.head()

,id,BeatsPerMinute
0,524164,119.035
1,524165,119.035
2,524166,119.035
3,524167,119.035
4,524168,119.035


In [7]:
TRAIN_CSV = "train.csv"
TEST_CSV  = "test.csv"
TARGET = "BeatsPerMinute"
IDCOL = "id"

In [8]:
DEFAULT_PARAMS = {
    "learning_rate": 0.05,
    "n_estimators": 1000, 
    "num_leaves": 63,
    "min_child_samples": 50,
    "reg_lambda": 1.0,
    "reg_alpha": 0.0,
    "early_stopping_rounds": 50,
    "verbose": -1,
    "random_state": 42
}
SKLEARN_PARAMS = {
    "learning_rate": 0.05,
    "max_iter": 400,
    "max_leaf_nodes": 63,
    "min_samples_leaf": 50,
    "early_stopping": True,
    "validation_fraction": 0.1,
    "random_state": 42
}

In [9]:
TUNE_SUBSAMPLE = 100_000
def add_engineered_features(df):
    """Add a handful of robust features derived from the dataset's numeric columns."""
    df = df.copy()
    # duration in seconds
    if "TrackDurationMs" in df.columns:
        df["Duration_s"] = df["TrackDurationMs"] / 1000.0
        df["log_Duration_s"] = np.log1p(df["Duration_s"])
    # interactions that make musical sense
    if "RhythmScore" in df.columns and "Energy" in df.columns:
        df["Rhythm_x_Energy"] = df["RhythmScore"] * df["Energy"]
    if "InstrumentalScore" in df.columns and "AcousticQuality" in df.columns:
        df["Instrumental_over_Acoustic"] = (df["InstrumentalScore"] + 1e-6) / (df["AcousticQuality"] + 1e-6)
    if "VocalContent" in df.columns and "InstrumentalScore" in df.columns:
        df["Vocal_to_Instrumental"] = (df["VocalContent"] + 1e-6) / (df["InstrumentalScore"] + 1e-6)
    if "MoodScore" in df.columns and "RhythmScore" in df.columns:
        df["Mood_x_Rhythm"] = df["MoodScore"] * df["RhythmScore"]
    return df

def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

In [10]:
X = train_data.drop(columns=[IDCOL, TARGET])
y = train_data[TARGET].values
X_test = test_data.drop(columns=[IDCOL])

X = add_engineered_features(X)
X_test = add_engineered_features(X_test)


for c in X.columns:
    if c not in X_test.columns:
        X_test[c] = 0
X_test = X_test[X.columns] 

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [12]:
# Complete LightGBM Model for Google Colab
# Just run this cell after loading your data (X_train, y_train, X_val, y_val)

import lightgbm as lgb
import optuna
import numpy as np
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": "gbdt",
        "device": "gpu",  # Changed to CPU for Colab compatibility
        "verbosity": -1,
        "seed": 42,
        
        # Optimized parameter ranges
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
    }
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)
    
    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dval],
        num_boost_round=1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=0)  # Silent
        ]
    )
    
    preds = model.predict(X_val, num_iteration=model.best_iteration)
    rmse = mean_squared_error(y_val, preds, squared=False)
    return rmse

# Run optimization
print("Starting hyperparameter optimization...")
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=42)
)

study.optimize(objective, n_trials=50)

print(f"Best RMSE: {study.best_value:.6f}")
print("Best parameters found:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Get final parameters
best_params = study.best_params.copy()
best_params.update({
    "objective": "regression",
    "metric": "rmse", 
    "boosting_type": "gbdt",
    "device": "cpu",
    "verbosity": -1,
    "seed": 42
})

# Train final model
print("\nTraining final model with best parameters...")
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)

final_model = lgb.train(
    best_params,
    dtrain,
    valid_sets=[dtrain, dval],
    valid_names=['train', 'val'],
    num_boost_round=1000,
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)
    ]
)



# Feature importance (optional - uncomment if needed)
# import matplotlib.pyplot as plt
# lgb.plot_importance(final_model, max_num_features=20, figsize=(10, 8))
# plt.tight_layout()
# plt.show()

print("\nModel training complete! Use 'final_model' for predictions.")
print("Example: predictions = final_model.predict(X_test)")

[I 2025-09-02 19:51:15,427] A new study created in memory with name: no-name-2a942936-d568-4132-9ce0-1dc3cf049a41


Starting hyperparameter optimization...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 50 rounds


[I 2025-09-02 19:51:24,231] Trial 0 finished with value: 26.444369200535657 and parameters: {'learning_rate': 0.03574712922600244, 'num_leaves': 287, 'max_depth': 10, 'feature_fraction': 0.759195090518222, 'bagging_fraction': 0.4936111842654619, 'bagging_freq': 2, 'min_child_samples': 10, 'lambda_l1': 0.6245760287469893, 'lambda_l2': 0.002570603566117598, 'min_split_gain': 0.7080725777960455, 'reg_alpha': 1.5320059381854043e-08, 'reg_lambda': 5.360294728728285}. Best is trial 0 with value: 26.444369200535657.


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[158]	valid_0's rmse: 26.4384


[I 2025-09-02 19:54:22,532] Trial 48 finished with value: 26.43838783424946 and parameters: {'learning_rate': 0.011215636726196954, 'num_leaves': 34, 'max_depth': 8, 'feature_fraction': 0.8260364702903364, 'bagging_fraction': 0.7625711395315482, 'bagging_freq': 3, 'min_child_samples': 89, 'lambda_l1': 3.474639330107848, 'lambda_l2': 0.0008320983671211298, 'min_split_gain': 0.19502568133867274, 'reg_alpha': 2.3445748754199203, 'reg_lambda': 7.785620212593411e-06}. Best is trial 46 with value: 26.437615683345495.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[109]	valid_0's rmse: 26.4383


[I 2025-09-02 19:54:28,205] Trial 49 finished with value: 26.438349243798026 and parameters: {'learning_rate': 0.011556614280545358, 'num_leaves': 54, 'max_depth': 9, 'feature_fraction': 0.8466619949292421, 'bagging_fraction': 0.7782159371910851, 'bagging_freq': 2, 'min_child_samples': 80, 'lambda_l1': 0.6750029922581606, 'lambda_l2': 8.837739435286319e-05, 'min_split_gain': 0.027348602305910172, 'reg_alpha': 8.866509416948462, 'reg_lambda': 1.6929422816928325e-08}. Best is trial 46 with value: 26.437615683345495.


Best RMSE: 26.437616
Best parameters found:
  learning_rate: 0.01125640881199085
  num_leaves: 31
  max_depth: 9
  feature_fraction: 0.8375482070888639
  bagging_fraction: 0.7904281065979449
  bagging_freq: 3
  min_child_samples: 92
  lambda_l1: 2.1267912585010884
  lambda_l2: 0.0010722372697166448
  min_split_gain: 0.045034833982646405
  reg_alpha: 2.70071408461679
  reg_lambda: 1.0567766942564716e-05

Training final model with best parameters...
Training until validation scores don't improve for 50 rounds
[50]	train's rmse: 26.4583	val's rmse: 26.4402
[100]	train's rmse: 26.4476	val's rmse: 26.4384
[150]	train's rmse: 26.4388	val's rmse: 26.4381
[200]	train's rmse: 26.4304	val's rmse: 26.4378
[250]	train's rmse: 26.4224	val's rmse: 26.4381
Early stopping, best iteration is:
[201]	train's rmse: 26.4303	val's rmse: 26.4378

Model training complete! Use 'final_model' for predictions.
Example: predictions = final_model.predict(X_test)


In [13]:
# Make predictions
train_preds = final_model.predict(X_train, num_iteration=final_model.best_iteration)
val_preds = final_model.predict(X_val, num_iteration=final_model.best_iteration)

# Calculate final scores
train_rmse = mean_squared_error(y_train, train_preds, squared=False)
val_rmse = mean_squared_error(y_val, val_preds, squared=False)

print(f"\nFinal Results:")
print(f"Training RMSE: {train_rmse:.6f}")
print(f"Validation RMSE: {val_rmse:.6f}")


Final Results:
Training RMSE: 26.430259
Validation RMSE: 26.437803


In [14]:
# best_params = study.best_params
# best_params.update({
#     "objective": "regression",
#     "metric": "rmse",
#     "boosting_type": "gbdt",
#     "device": "gpu"
# })

# final_model = lgb.train(
#     best_params,
#     lgb.Dataset(X_train, label=y_train),
#     valid_sets=[lgb.Dataset(X_val, label=y_val)],
#     num_boost_round=1000,
#     callbacks=[
#         early_stopping(stopping_rounds=100),
#         log_evaluation(period=100)
#     ]
# )


In [15]:
preds_test = final_model.predict(X_test, num_iteration=final_model.best_iteration)

In [16]:
submission = pd.DataFrame({
    "id": test_data["id"],         
    "BeatsPerMinute": preds_test             
})

In [17]:
# submission.to_csv("sample_submission.csv", index=False)
# print("✅ sample_submission.csv created!")

In [18]:
submission.head()

,id,BeatsPerMinute
0,524164,118.973859
1,524165,119.077959
2,524166,119.669772
3,524167,119.429488
4,524168,119.297035


In [19]:
sample.head()

,id,BeatsPerMinute
0,524164,119.035
1,524165,119.035
2,524166,119.035
3,524167,119.035
4,524168,119.035


In [20]:
# final_model.save_model('model.h5')